In [1]:
# BASE GAME
import sys
import os
import json
import pygame as pg
import numpy as np
import NeuralNetwork as nn


# game class
class FlappyJuan:
    def __init__(self, w_, h_):
        self.w = w_ # width
        self.h = h_ # height
        self.window = pg.display.set_mode((w_, h_)) #pygame window
        self.speed = 2 # speed of scrolling
        self.state = 0 # game state: start, playing, game-over
        self.score = 0 # pipes passed
        self.highScore = 0 # highescore of run
        self.counted = False # helper variable for acurate pipe counting
        self.juan = Juan(w_, h_) # player
        self.fileName = "./Resources/BestJuanJson.txt" # file path for json of weights
        self.AI = JuanAI(w_, h_, self.fileName) # load AI juan from the given file path
        self.showAI = True # whether or not to draw the AI Juan
        self.sarahs = [Pipe(self.w,
                            np.random.randint(150, 450)), 
                       Pipe(self.w + 300, 
                            np.random.randint(150, 450)),
                      Pipe(self.w + 600, 
                            np.random.randint(150, 450))] # obstacles
            
    # reset game
    def reset(self):
        self.score = 0
        self.counted = False
        self.juan = Juan(self.w, self.h)
        self.AI = JuanAI(self.w, self.h, self.fileName)
        self.sarahs = [Pipe(self.w, 
                            np.random.randint(150, 450)), 
                       Pipe(self.w + 300, 
                            np.random.randint(150, 450)),
                      Pipe(self.w + 600, 
                            np.random.randint(150, 450))]
    
    # update player, obstacles, and score
    def update(self):
        if (self.juan.collide(self)):
            self.state = 2
        self.juan.update()
        self.AI.think(self)
        self.AI.update()
        for sarah in self.sarahs:
            sarah.update(self.speed)
        if self.nextPipes()[0].rect.x < (self.w * 4 / 13) and not self.counted:
            self.score += 1
            self.counted = True
        if self.sarahs[0].rect.x < -100:
            self.sarahs.pop(0)
            self.sarahs.append(Pipe(self.w + 254, 
                                    np.random.randint(150, 450)))
            self.counted = False
            
    # returns closest 2 pipes in front of the player
    def nextPipes(self):
        closestIndex = None
        closestDist = float('inf')
        for i in range(len(self.sarahs)):
            sarah = self.sarahs[i]
            d = sarah.rect.x - (self.w * 4 / 13)
            if d < closestDist and d > - sarah.sarah.get_width():
                closestIndex = i
                closestDist = d
        return [self.sarahs[closestIndex], self.sarahs[closestIndex + 1]]
    
    # draw and display game
    def draw(self):
        self.window.fill('white')
        font = pg.font.SysFont(None, 30)
        text = font.render("Score: " + str(self.score), True, 'red')
        align = text.get_rect(topleft=(10, 10))
        self.window.blit(text, align)
        text = font.render("High Score: " + str(self.highScore), True, 'red')
        align = text.get_rect(topleft=(10, 40))
        self.window.blit(text, align)
        if self.showAI:
            self.AI.draw(self.window)
        self.juan.draw(self.window)
        for sarah in self.sarahs:
            sarah.draw(self.window)
    
    # methods to run every tick
    def onTick(self):
        if self.state == 1:
            self.draw()
            self.update()
        elif self.state == 0:
            self.startScreen()
        else: 
            self.endScreen()

    # display start screen
    def startScreen(self):
        self.window.fill((255, 255, 255))
        font = pg.font.SysFont(None, 50)
        text = font.render("SPACE BAR TO PLAY", True, 'black')
        textCenter = text.get_rect(center=(self.w / 2, self.h * 2 / 3))
        self.window.blit(text, textCenter)
        font = pg.font.SysFont(None, 25)
        text = font.render("PRESS ENTER TO TOGGLE GHOST", True, 'gray')
        textCenter = text.get_rect(center=(self.w / 2, self.h * 5 / 6))
        self.window.blit(text, textCenter)
        
    # display end screen
    def endScreen(self):
        self.startScreen()
        font = pg.font.SysFont(None, 50)
        endText = font.render("GAME OVER", True, 'red')
        endCenter = endText.get_rect(center=(self.w / 2, self.h / 3))
        self.window.blit(endText, endCenter)
        scoreText = font.render(str(self.score), True, 'red')
        scoreCenter = scoreText.get_rect(center=(self.w / 2, self.h / 2))
        self.window.blit(scoreText, scoreCenter)

    # pygame run loop
    def play(self):
        pg.init()
        pg.font.init()
        while True:
            self.highScore = max(self.score, self.highScore)
            for event in pg.event.get():
                if event.type == pg.QUIT:
                    pg.quit()
                    sys.exit()
                elif event.type == pg.KEYDOWN:
                    if event.key == pg.K_RETURN:
                        self.showAI ^= 1
                    if event.key == pg.K_SPACE:
                        if self.state == 1:
                            self.juan.flap()
                        else:
                            self.state = 1
                            self.reset()
            self.onTick()
            pg.display.update()
            pg.time.Clock().tick(60)

            
# pipe class
class Pipe(pg.sprite.Sprite):
    def __init__(self, x_, y_):
        super().__init__() # pygame sprite constructor
        self.sarahT = pg.transform.rotozoom(pg.image.load(
            './Resources/PipeySarah.png'), 180, 0.4) # top pipe image
        self.sarahB = pg.transform.rotozoom(pg.image.load(
            './Resources/PipeySarah.png'), 0, 0.4) # bottom pipe image
        self.gap = np.random.randint(175, 250) # gap size between top and bottom
        self.sarah = self.makePipe().convert_alpha() # combined pipe image
        self.rect = self.sarah.get_rect(center=(x_, y_)) # rect.x and .y hold coords of pipe
        self.mask = pg.mask.from_surface(self.sarah) # mask of pipe for collisions
    
    # combines the two pipe images with given gap dist apart
    def makePipe(self):
        pipe = pg.Surface((self.sarahB.get_width(), 
                           2 * self.sarahB.get_height() + self.gap), 
                          pg.SRCALPHA)
        pipe.blit(self.sarahT, (0, 0))
        pipe.blit(self.sarahB, (0, self.sarahB.get_height() + self.gap))
        return pipe
    
    # drawing the pipe into the scene
    def draw(self, surface):
        surface.blit(self.sarah, self.rect)
    
    # moving the pipe
    def update(self, speed):
        self.rect.x -= speed
        
        
# player class
class Juan(pg.sprite.Sprite):
    def __init__(self, w_, h_ ):
        super().__init__() # pygame sprite constructor
        self.acc = 0 # acceleration
        self.vel = -1 # velocity
        self.angle = 0 # angle
        self.gravity = 0.5 # strength of gravity
        self.juan = pg.transform.rotozoom(pg.image.load('./Resources/FlappyJuan.png'), 0, 0.3) # player image
        self.rect = self.juan.get_rect(center=(w_ * 4 / 13, h_ / 4)) # rect.x and .y hold coords of pipe
        self.mask = pg.mask.from_surface(self.juan) # mask of pipe for collisions
        
    # drawing the player into the scene
    def draw(self, surface):
        surface.blit(pg.transform.rotate(self.juan, self.angle), self.rect)
        
    # apply force, physics
    def applyForce(self, force):
        self.acc += force
        
    # moving the player, physics
    def update(self):
        self.applyForce(self.gravity);
        if self.rect.y >= 0:
            self.vel += self.acc;
            self.angle = -self.vel
            self.rect.y += self.vel;
        else:
            self.rect.y = 0
            self.vel = 0
        self.acc = 0
        
    # apply upward flap force
    def flap(self):
        self.applyForce(-30 * self.gravity)
        
    # test for collision with first pipe
    def collide(self, f):
        return self.rect.y + self.juan.get_height() > f.h or \
        pg.sprite.spritecollide(self, 
                                pg.sprite.GroupSingle(f.nextPipes()[0]), 
                                False, 
                                pg.sprite.collide_mask)
        

# AI class
class JuanAI(Juan):
    def __init__(self, w_, h_, fileName):
        super().__init__(w_, h_) # Juan class Constructor
        self.brain = self.loadBestJuan(fileName)
        
    # take in inputs: self y, dist from pipe, pipe y, pipe gap
    def think(self, f):
        pipes = f.nextPipes()
        p1 = pipes[0]
        inputs = np.array([[self.rect.y / f.h, 
                            self.vel / 100,
                            p1.rect.x / f.w,
                            (p1.rect.y + (p1.gap / 2)) / f.h, 
                            (p1.rect.y - (p1.gap / 2)) / f.h]])
        outputs = self.brain.feedForward(inputs)
        if outputs[0][0] > outputs[0][1]:
            self.applyForce(-30 * self.gravity)
        
    # load the weights from the saved best juan.brain
    def loadBestJuan(self, fileName):
        if os.path.exists(fileName):
            with open(fileName) as f:
                data = json.load(f)
            brain = nn.NeuralNetwork(5, 12, 2, 0.1)
            brain.wI = np.array(data["wI"])
            brain.wO = np.array(data["wO"])
            brain.biasH = np.array(data["biasH"])
            brain.biasO = np.array(data["biasO"])
            self.juan.set_alpha(100)
        return brain
        
        
# initiate and run
fj = FlappyJuan(600, 600)
fj.play()

pygame 2.1.2 (SDL 2.0.18, Python 3.8.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

/Users/aidan/Library/Python/3.8/lib/python/site-packages/IPython/core/interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
### NEUROEVOLUTION
import sys
import os
import json
import pygame as pg
import numpy as np
import NeuralNetwork as nn


# game class
class FlappyJuan:
    def __init__(self, w_, h_, popSize_):
        self.w = w_ # width
        self.h = h_ # height
        self.window = pg.display.set_mode((w_, h_)) #pygame window
        self.speed = 2 # speed of scrolling
        self.score = 0 # pipes passed
        self.counted = False # helper variable for acurate pipe counting
        self.popSize = popSize_ # size of each generation
        self.gen = 0 # generation number
        self.numAlive = popSize_ # number of juans that are not collided, starts at full pop
        self.sarahs = [Pipe(self.w, 
                            np.random.randint(150, 450)), 
                       Pipe(self.w + 300, 
                            np.random.randint(150, 450)),
                      Pipe(self.w + 600, 
                            np.random.randint(150, 450))] # obstacles
        self.juans = self.generatePopulation() # list of birds in current population
        self.highScore = 0 # highest score of current run
        self.bestOnly = False # only draw current juan with best fitness
        self.fileName = "./Resources/BestJuanJson.txt" # file path to write best juan data to
        self.bestEver = self.loadBestJuan() # best ever recorded juan

    # reset game
    def reset(self):
        self.score = 0
        self.gen += 1
        self.counted = False
        self.sarahs = [Pipe(self.w, 
                            np.random.randint(150, 450)), 
                       Pipe(self.w + 300, 
                            np.random.randint(150, 450)),
                       Pipe(self.w + 600, 
                            np.random.randint(150, 450))]
        self.juans = self.generatePopulation(self.juans)
        self.bestEver = self.loadBestJuan()

    # make new random population
    def generatePopulation(self, prev=None):
        pop = []
        if prev is None:
            for i in range(self.popSize):
                pop.append(Juan(self.w, self.h))
        else: 
            total = 0
            for juan in prev:
                total += juan.score
            for juan in prev:
                juan.score /= total
            for i in range(self.popSize):
                pop.append(self.pickChild())
        return pop
    
    # picks child from population based on fitness
    def pickChild(self):
        index = 0;
        r = np.random.rand()
        while (r > 0):
            r = r - self.juans[index].score
            index += 1
        index -= 1
        child = Juan(self.w, self.h, self.juans[index])
        child.brain.mutate(0.25, 0.1)
        return child
        
    # update player, obstacles, and score
    def update(self, runSpeed):           
        for i in range(runSpeed):
            self.numAlive = self.popSize
            for sarah in self.sarahs:
                sarah.update(self.speed)
            if self.nextPipes()[0] != self.sarahs[0] and not self.counted:
                self.score += 1 
                self.counted = True
            if self.sarahs[0].rect.x < -100:
                self.sarahs.pop(0)
                self.sarahs.append(Pipe(self.w + 254, 
                                        np.random.randint(150, 450)))
                self.counted = False
            self.bestEver.update(self)
            for juan in self.juans:
                juanScore = juan.update(self)
                if juan.collided:
                    self.numAlive -= 1
            if self.numAlive == 0 and (not self.bestOnly or self.bestEver.collided):
                self.reset()                
            
    # returns closest 2 pipes in front of the player
    def nextPipes(self):
        closestIndex = None
        closestDist = float('inf')
        for i in range(len(self.sarahs)):
            sarah = self.sarahs[i]
            d = sarah.rect.x - (self.w * 4 / 13)
            if d < closestDist and d > - sarah.sarah.get_width():
                closestIndex = i
                closestDist = d
        return [self.sarahs[closestIndex], self.sarahs[closestIndex + 1]]
    
    # draw and display game
    def draw(self):
        self.window.fill('white')
        font = pg.font.SysFont(None, 30)
        text = font.render("Score: " + str(self.score), True, 'red')
        align = text.get_rect(topleft=(10, 10))
        self.window.blit(text, align)
        text = font.render("High Score: " + str(self.highScore), True, 'red')
        align = text.get_rect(topleft=(10, 40))
        self.window.blit(text, align)
        text = font.render("Generation: " + str(self.gen), True, 'red')
        align = text.get_rect(topleft=(10, 70))
        self.window.blit(text, align)
        text = font.render("Alive: " + str(self.numAlive), True, 'red')
        align = text.get_rect(topleft=(10, 100))
        self.window.blit(text, align)
        self.bestEver.draw(self.window) # draw best juan
        if not self.bestOnly: 
            for juan in self.juans: # draw all juans
                juan.draw(self.window)
        for sarah in self.sarahs:
            sarah.draw(self.window)

    # methods to run every tick
    def onTick(self, runSpeed):   
        self.update(runSpeed)
        self.draw()

    # pygame run loop
    def play(self):
        pg.init()
        pg.font.init()
        clock = pg.time.Clock()
        runSpeed = 1
        while True:
            if self.score > self.highScore and self.numAlive > 0:
                self.highScore = self.score
                self.saveJuan()
            for event in pg.event.get():
                if event.type == pg.QUIT:
                    self.saveJuan()
                    pg.quit()
                    sys.exit()
                elif event.type == pg.KEYDOWN:
                    if event.key == pg.K_SPACE: # toggle showing best only or all
                        self.bestOnly ^= 1 # best only runs significantly smoother
                    if event.key == pg.K_RETURN:
                        self.reset()
                    # speed up and slow down generations
                    elif event.key == pg.K_1:
                        runSpeed = 1
                    elif event.key == pg.K_2:
                        runSpeed = 5
                    elif event.key == pg.K_3:
                        runSpeed = 20
                    elif event.key == pg.K_4:
                        runSpeed = 100
            self.onTick(runSpeed)
            pg.display.update()
            clock.tick(60)
            
    # save highscore to json if the current highscore is higher than the saved one
    def saveJuan(self):
        newScore = True
        if os.path.exists(self.fileName):
            with open(self.fileName, "r") as f:
                newScore = newScore and fj.highScore > json.load(f)["score"]
        if newScore:
            maxScore = 0
            maxJuan = None
            for juan in self.juans:
                if juan.score > maxScore:
                    maxScore = juan.score
                    maxJuan = juan
            self.writeJsonJuan(maxJuan)

                    
    # writes given juan to json file
    def writeJsonJuan(self, maxJuan):
        juanInfo = {
            "score" : self.highScore,
            "wI" : maxJuan.brain.wI.tolist(),
            "wO" : maxJuan.brain.wO.tolist(),
            "biasH" : maxJuan.brain.biasH.tolist(),
            "biasO" : maxJuan.brain.biasO.tolist()
        }
        with open(self.fileName, "w") as f:
            json.dump(juanInfo, f)
                    
    # load the weights from the saved best juan.brain
    def loadBestJuan(self):
        bestJuan = Juan(self.w, self.h)
        if os.path.exists(self.fileName):
            with open(self.fileName) as f:
                data = json.load(f)
            bestJuan.brain.wI = np.array(data["wI"])
            bestJuan.brain.wO = np.array(data["wO"])
            bestJuan.brain.biasH = np.array(data["biasH"])
            bestJuan.brain.biasO = np.array(data["biasO"])
            bestJuan.juan.set_alpha(100)
        return bestJuan
    
               
# pipe class
class Pipe(pg.sprite.Sprite):
    def __init__(self, x_, y_):
        super().__init__() # pygame sprite constructor
        self.sarahT = pg.transform.rotozoom(pg.image.load(
            './Resources/PipeySarah.png'), 180, 0.4) # top pipe image
        self.sarahB = pg.transform.rotozoom(pg.image.load(
            './Resources/PipeySarah.png'), 0, 0.4) # bottom pipe image
        self.gap = np.random.randint(175, 250) # gap size between top and bottom
        self.sarah = self.makePipe().convert_alpha() # combined pipe image
        self.rect = self.sarah.get_rect(center=(x_, y_)) # rect.x and .y hold coords of pipe
        self.mask = pg.mask.from_surface(self.sarah) # mask of pipe for collisions
    
    # combines the two pipe images with given gap dist apart
    def makePipe(self):
        pipe = pg.Surface((self.sarahB.get_width(), 
                           2 * self.sarahB.get_height() + self.gap), 
                          pg.SRCALPHA)
        pipe.blit(self.sarahT, (0, 0))
        pipe.blit(self.sarahB, (0, self.sarahB.get_height() + self.gap))
        return pipe
    
    # drawing the pipe into the scene
    def draw(self, surface):
        surface.blit(self.sarah, self.rect)
    
    # moving the pipe
    def update(self, speed):
        self.rect.x -= speed
        
        
# population member class
class Juan(pg.sprite.Sprite):
    def __init__(self, w_, h_, clone=None):
        super().__init__() # pygame sprite constructor
        self.acc = 0.0 # acceleration
        self.vel = -1.0 # velocity
        self.angle = 0.0 # angle
        self.gravity = 0.5 # strength of gravity
        self.collided = False
        self.juan = pg.transform.rotozoom(pg.image.load(
            './Resources/FlappyJuan.png'), 0, 0.3).convert_alpha() # player image
        self.rect = self.juan.get_rect(center=(w_ * 4 / 13, h_ / 4)) # rect.x and .y hold coords of pipe
        self.mask = pg.mask.from_surface(self.juan) # mask of pipe for collisions
        self.score = 0.0 # fitness score, higher = better
        if clone is None:
            self.brain = nn.NeuralNetwork(5, 12, 2, 0.1) # dna object in form of Neural Network
        else:
            self.brain = clone.brain.copy()
        
    # drawing the player into the scene
    def draw(self, surface):
        surface.blit(pg.transform.rotate(self.juan, self.angle), self.rect)

        
    # apply force, physics
    def applyForce(self, force):
        self.acc += force
        
    # moving the player, physics
    def update(self, f):
        if self.collide(f) or self.collided:
            self.collided = True
            self.rect.x -= f.speed
            return 0
        else:
            self.applyForce(self.gravity);
            self.think(f)
            if self.rect.y > f.h / 4 and self.rect.y < f.h * 3 / 4:
                self.score += 10
            if self.rect.y >= 0:
                self.score += 1
                self.vel += self.acc;
                self.angle = -self.vel
                self.rect.y += self.vel;
            else:
                self.rect.y = 0.0
                self.vel = 0.0
            self.acc = 0.0
            return self.score
        
    # take in inputs: self y, dist from pipe, pipe y, pipe gap
    def think(self, f):
        pipes = f.nextPipes()
        p1 = pipes[0]
        inputs = np.array([[self.rect.y / f.h, 
                            self.vel / 100,
                            p1.rect.x / f.w,
                            (p1.rect.y + (p1.gap / 2)) / f.h, 
                            (p1.rect.y - (p1.gap / 2)) / f.h]])
        outputs = self.brain.feedForward(inputs)
        if outputs[0][0] > outputs[0][1]:
            self.applyForce(-30 * self.gravity)
        
    # test for collision with first pipe
    def collide(self, f):
        return self.rect.y + self.juan.get_height() > f.h or \
        pg.sprite.spritecollide(self, 
                                pg.sprite.GroupSingle(f.nextPipes()[0]), 
                                False, 
                                pg.sprite.collide_mask)
        
    
# initiate and run
fj = FlappyJuan(600, 600, 500) 
fj.play()

pygame 2.1.2 (SDL 2.0.18, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html
